<a href="https://colab.research.google.com/github/custom-hyper/DeFi_Lama_API/blob/master/DeFi_Lama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Setup

In [ ]:
#@title  Install  Packages { display-mode: "code" }
!pip install requests
!pip install json
!pip install psycopg2
!pip install pandas


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [ ]:
#@title Ta-lib – Extra steps { display-mode: "code" }
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

Selecting previously unselected package libta-lib0.
(Reading database ... 155501 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 269 kB 7.1 MB/s 
  Created wheel for ta-lib: filename=TA_Lib-0.4.24-cp37-cp37m-linux_x86_64.whl size=1465746 sha256=b1726f1c78d103c34c31a00d6564c5883201385b499d9d0230850c6dc46bb0b2
  Stored in directory: /root/.cache/pip/wheels/02/b1/a5/aca31e8cdd0137d7a83bf72237056b8705663dd1f9b5eac11e
Successfully built ta-lib


In [ ]:
#@title Connect to Drive { display-mode: "code" }
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title  Import Libraries { display-mode: "code" }
import requests
import numpy as np
import talib as ta
import json
from google.colab import data_table
import sqlite3 as sq
import pandas as pd
import logging
import pandas as pd
import logging
import time
from tqdm.notebook import tqdm
from datetime import datetime
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [ ]:
#@title  Connect SQLite{ display-mode: "code" }
path = '/content/drive/MyDrive/workspace/pipelines/db/' #@param {type: 'string'
database = "db.sqlite" #@param {type: 'string'
conn_SQLite = sq.connect(path+database)

#@title   Show Available Tables { display-mode: "code" }
cursor = conn_SQLite.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.execute("DROP TABLE deFi_lama_general;")
cursor.fetchall()

[]

# Data Extraction

Please use  df['slug'] as primary key if loaded into SQL light

## Protocols ID List

In [ ]:
#@title Call API { display-mode: "code" }



#General Information table
response = requests.get("https://api.llama.fi/protocols")
output = response.text
DeFi = json.loads(output)
full_talbe = pd.json_normalize(DeFi,max_level=4)
full_talbe['id'] = full_talbe['symbol']  + '.' + full_talbe['name']
full_talbe['id'].replace(' ','')
full_talbe['Notes'] = ''

#UNIX timestamp: created
full_talbe['UNIX'] = int( time.time())



full_talbe.head(1)

,id,name,address,symbol,url,description,chain,logo,audits,audit_note,...,chainTvls.zkSync,chainTvls.zkSync-offers,chainTvls.Metis-pool2,chainTvls.Harmony-Treasury,chainTvls.Metis-Treasury,chainTvls.Velas-staking,chainTvls.Celo-Treasury,chainTvls.Ergo-staking,Notes,UNIX
0,CRV.Curve,Curve,0xD533a949740bb3306d119CC777fa900bA034cd52,CRV,https://curve.fi,Curve is a decentralized exchange liquidity po...,Multi-Chain,https://icons.llama.fi/curve.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1650754198


In [ ]:
#@title Save To Directory{ display-mode: "code" }
table = "deFi_lama_general"
sfile = "deFi_lama_general.csv"
path = "/content/drive/MyDrive/workspace/pipelines/datasets/defi_lama/output/"
full_talbe.to_csv(path + sfile)

NameError: ignored

In [ ]:
#@title Load to SQLite { display-mode: "code" }


full_talbe.to_sql(table, conn_SQLite , if_exists='append', index=False) 

df = pd.read_sql('select * from {}'.format(sfile), conn_SQLite)


InterfaceError: ignored

In [ ]:
full_talbe


,id,name,address,symbol,url,description,chain,logo,audits,audit_note,...,chainTvls.zkSync,chainTvls.zkSync-offers,chainTvls.Metis-pool2,chainTvls.Harmony-Treasury,chainTvls.Metis-Treasury,chainTvls.Velas-staking,chainTvls.Celo-Treasury,chainTvls.Ergo-staking,Notes,UNIX
0,CRV.Curve,Curve,0xD533a949740bb3306d119CC777fa900bA034cd52,CRV,https://curve.fi,Curve is a decentralized exchange liquidity po...,Multi-Chain,https://icons.llama.fi/curve.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1650757207
1,LDO.Lido,Lido,0x5a98fcbea516cf06857215779fd812ca3bef1b32,LDO,https://lido.fi/,"Liquidity for staked assets. Daily rewards, no...",Multi-Chain,https://icons.llama.fi/lido.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1650757207
2,ANC.Anchor,Anchor,-,ANC,https://anchorprotocol.com/,Money market on Terra,Multi-Chain,https://icons.llama.fi/anchor.jpg,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1650757207
3,MKR.MakerDAO,MakerDAO,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,MKR,https://makerdao.com/,"Builders of Dai, a digital currency that can b...",Ethereum,https://icons.llama.fi/makerdao.jpg,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1650757207
4,CVX.Convex Finance,Convex Finance,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b,CVX,https://www.convexfinance.com/,Convex simplifies your Curve-boosting experien...,Ethereum,https://icons.llama.fi/convex-finance.jpg,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1650757207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1510,PUSH.EPNS,EPNS,0xf418588522d5dd018b425E472991E52EBBeEEEEE,PUSH,https://epns.io,The native communication layer of Web3. Send /...,Ethereum,https://icons.llama.fi/epns.jpg,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1650757207
1511,TNODE.Trusted Node,Trusted Node,bsc:0x7f12A37B6921FFac11FAB16338B3ae67EE0c462b,TNODE,https://trustednode.io,Tnode token ables you to: Participate in the g...,Multi-Chain,https://icons.llama.fi/trusted-node.png,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1650757207
1512,LION.Scrub Finance,Scrub Finance,cronos:0x49fB98F9b4a3183Cd88e7a115144fdf00fa6fB95,LION,https://scrub.finance,Scrub Finance is the first deflationary algori...,Cronos,https://icons.llama.fi/scrub-finance.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1650757207
1513,SCNR.Swapscanner,Swapscanner,klaytn:0x8888888888885b073f3c81258c27e83db228d5f3,SCNR,https://swapscanner.io,Swapscanner is a next-generation DEX Aggregato...,Klaytn,https://icons.llama.fi/swapscanner.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1650757207


In [ ]:
#@title Show Stats { display-mode: "code" }
full_talbe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1515 entries, 0 to 1514
Columns: 305 entries, id to UNIX
dtypes: float64(279), int64(1), object(25)
memory usage: 3.5+ MB


 ## TVL Timeseries

In [ ]:
#@title Call API  { display-mode: "code" }

#General Information table
response = requests.get("https://api.llama.fi/protocols")
output = response.text
table = json.loads(output)
df = pd.json_normalize(table)
slug = df['slug']

path = "https://api.llama.fi/protocol/"
frames = []


try:
  for i in tqdm(slug):
    response = requests.get(path + i)
    output = response.text
    DeFi = json.loads(output)

    full_table = pd.json_normalize(DeFi['tvl']) 
    full_table['slug'] = i
    

    #results = [full_table]
    frames.append(full_table)

    #Slow down requests
    
    time.sleep(60/50)
    #Status report
    
    
except ConnectionError:
    time.sleep(60)


single = pd.concat(frames) #Careful, this line must be outside of the for loop
#single['date_Standard']  = datetime.fromtimestamp(time).strftime('%Y-%m-%d')


single.head()

  


  0%|          | 0/1515 [00:00<?, ?it/s]

,date,totalLiquidityUSD,slug
0,1581206400,1.163733e+06,curve
1,1581292800,1.372110e+06,curve
2,1581379200,9.607534e+05,curve
3,1581465600,1.006408e+06,curve
4,1581552000,1.258776e+06,curve


In [ ]:
#@title Save To Directory{ display-mode: "code" }
sfile = "deFi_lama_tvl.csv"
path = "/content/drive/MyDrive/workspace/pipelines/datasets/defi_lama/output/"
single.to_csv(path + sfile)

In [ ]:
#@title Load to SQLite { display-mode: "code" }

table = "deFi_lama_tvl"

single.to_sql(table, conn_SQLite , if_exists='replace', index=False) # writes to file

 # Test Connection
df = pd.read_sql('select * from {}'.format(table), conn_SQLite)

df.head()


In [ ]:
#@title Show Stats { display-mode: "code" }
single.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 290154 entries, 0 to 3
Data columns (total 3 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   date               290154 non-null  int64  
 1   totalLiquidityUSD  290154 non-null  float64
 2   slug               290154 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 8.9+ MB


# Data Wrangling

In [ ]:
table = "deFi_lama_tvl"
df = pd.read_sql('select * from {}'.format(table), conn_SQLite)

df.head()


,date,totalLiquidityUSD,slug
0,1581206400,1.163733e+06,curve
1,1581292800,1.372110e+06,curve
2,1581379200,9.607534e+05,curve
3,1581465600,1.006408e+06,curve
4,1581552000,1.258776e+06,curve


In [ ]:
data = df
token_list = data['slug'].unique()
frames = []
period = 'd'

#@title Generate Indicators { display-mode: "code" }
for  i in token_list:

 # data_token = data.loc('CoinGecko_id'str(i))
  data_token = data[data['slug'] == i]
  
	#return
  re_day = [1, 7, 30, 90, 180, 365]
  for ma in re_day:
    column_name = "return_{}_{}".format(ma,period)
    data_token[column_name] =  np.log(data_token['totalLiquidityUSD'] / data_token['totalLiquidityUSD'].shift(ma))

  # Standard Deviation, Variance
  timeperiod_list = [ 5, 15,30,90]
  for timeperiod in timeperiod_list:
    stddev = "std_dev_{}_{}".format(timeperiod,period) 
    var = "var_{}_{}".format(timeperiod,period) 

    data_token[stddev] = ta.STDDEV(data_token['totalLiquidityUSD'], timeperiod=timeperiod, nbdev=1)    
    data_token[var] = ta.VAR(data_token['totalLiquidityUSD'], timeperiod=timeperiod, nbdev=1)    



  frames.append(data_token)



single = pd.concat(frames) #Careful, this line must be outside of the for loop
single.describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

,date,totalLiquidityUSD,return_1_d,return_7_d,return_30_d,return_90_d,return_180_d,return_365_d,std_dev_5_d,var_5_d,std_dev_15_d,var_15_d,std_dev_30_d,var_30_d,std_dev_90_d,var_90_d
count,2.901540e+05,2.901540e+05,2.686910e+05,2.609600e+05,2.318610e+05,1.636870e+05,9.100700e+04,2.433400e+04,2.841050e+05,2.841050e+05,2.692280e+05,2.692280e+05,2.476070e+05,2.476070e+05,1.707360e+05,1.707360e+05
mean,1.636924e+09,2.926728e+08,NaN,NaN,NaN,NaN,NaN,NaN,1.094959e+07,3.081670e+15,2.224315e+07,9.879373e+15,3.472677e+07,2.096053e+16,8.345512e+07,8.489318e+16
std,1.303841e+07,1.391264e+09,NaN,NaN,NaN,NaN,NaN,NaN,5.442230e+07,5.223012e+16,9.687440e+07,1.092146e+17,1.405513e+08,2.004162e+17,2.791574e+08,5.802694e+17
min,1.541203e+09,0.000000e+00,-inf,-inf,-inf,-inf,-inf,-inf,0.000000e+00,-1.855078e+02,0.000000e+00,-1.033333e+01,0.000000e+00,-1.575000e+02,0.000000e+00,-1.541528e+02
25%,1.631405e+09,4.590668e+05,-2.328424e-02,-1.003779e-01,-3.183222e-01,-7.517520e-01,-9.002057e-01,-4.956823e-01,1.174363e+04,1.379129e+08,3.695955e+04,1.366009e+09,7.948411e+04,6.317724e+09,4.382823e+05,1.920914e+11
50%,1.640736e+09,6.675922e+06,0.000000e+00,-6.069993e-03,-5.191016e-02,-1.721383e-01,-2.870973e-03,1.057302e+00,2.270672e+05,5.155952e+10,5.944331e+05,3.533507e+11,1.124045e+06,1.263477e+12,4.092318e+06,1.674706e+13
75%,1.646179e+09,6.083809e+07,2.110627e-02,7.235885e-02,1.748014e-01,3.988053e-01,9.296866e-01,2.762938e+00,2.387455e+06,5.699940e+12,5.801258e+06,3.365460e+13,1.027528e+07,1.055814e+14,3.218329e+07,1.035764e+15
max,1.650757e+09,2.429799e+10,inf,inf,inf,inf,inf,inf,2.799381e+09,7.836536e+18,2.524846e+09,6.374849e+18,3.002411e+09,9.014472e+18,4.036219e+09,1.629106e+19


In [ ]:
single.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 290154 entries, 0 to 290153
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   date               290154 non-null  int64  
 1   totalLiquidityUSD  290154 non-null  float64
 2   slug               290154 non-null  object 
 3   return_1_d         268691 non-null  float64
 4   return_7_d         260960 non-null  float64
 5   return_30_d        231861 non-null  float64
 6   return_90_d        163687 non-null  float64
 7   return_180_d       91007 non-null   float64
 8   return_365_d       24334 non-null   float64
 9   std_dev_5_d        284105 non-null  float64
 10  var_5_d            284105 non-null  float64
 11  std_dev_15_d       269228 non-null  float64
 12  var_15_d           269228 non-null  float64
 13  std_dev_30_d       247607 non-null  float64
 14  var_30_d           247607 non-null  float64
 15  std_dev_90_d       170736 non-null  float64
 16  va

## Protocol General Information Data

In [ ]:
#@title Call API  { display-mode: "code" }


#General Information table
response = requests.get("https://api.llama.fi/protocols")
output = response.text
DeFi = json.loads(output)
full_talbe = pd.json_normalize(DeFi)
full_talbe['id'] = full_talbe['symbol']  + '.' + full_talbe['name']
full_talbe['id'].replace(' ','')
full_talbe['Notes'] = ''

#UNIX timestamp: created
full_talbe['UNIX'] = int( time.time())



full_talbe.head(1)



,id,name,address,symbol,url,description,chain,logo,audits,audit_note,...,chainTvls.zkSync,chainTvls.zkSync-offers,chainTvls.Metis-pool2,chainTvls.Harmony-Treasury,chainTvls.Metis-Treasury,chainTvls.Velas-staking,chainTvls.Celo-Treasury,chainTvls.Ergo-staking,Notes,UNIX
0,CRV.Curve,Curve,0xD533a949740bb3306d119CC777fa900bA034cd52,CRV,https://curve.fi,Curve is a decentralized exchange liquidity po...,Multi-Chain,https://icons.llama.fi/curve.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1650813841


In [ ]:
 #@title Save To Directory{ display-mode: "code" }
 
table = "deFi_lama_general"
path = "/content/drive/MyDrive/workspace/pipelines/datasets/defi_lama/output/"
full_talbe.to_csv(path+ table)

In [ ]:
#@title Dataset Stats { display-mode: "code" }
full_talbe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1515 entries, 0 to 1514
Columns: 305 entries, id to UNIX
dtypes: float64(279), int64(1), object(25)
memory usage: 3.5+ MB


In [ ]:
#@title Load to SQLite { display-mode: "code" }

full_talbe.to_sql(table, conn , if_exists='replace', index=False) # writes to file

 # Test Connection
df = pd.read_sql('select * from {}'.format(table), conn)

df.head()

NameError: ignored

In [ ]:
full_talbe.describe()

,tvl,change_1h,change_1d,change_7d,staking,fdv,mcap,chainTvls.Optimism,chainTvls.Avalanche,chainTvls.Harmony,...,chainTvls.Arbitrum-offers,chainTvls.zkSync,chainTvls.zkSync-offers,chainTvls.Metis-pool2,chainTvls.Harmony-Treasury,chainTvls.Metis-Treasury,chainTvls.Velas-staking,chainTvls.Celo-Treasury,chainTvls.Ergo-staking,UNIX
count,1.517000e+03,1324.000000,1305.000000,1277.000000,6.930000e+02,6.360000e+02,1.047000e+03,3.300000e+01,2.010000e+02,5.100000e+01,...,1.000,1.0,1.000000,3.000000e+00,1.000000,1.000000,2.000000e+00,1.0,1.000000e+00,1.517000e+03
mean,1.646556e+08,-0.043550,0.023418,14.263569,3.178178e+07,1.129484e+12,9.472122e+07,1.586235e+07,5.617614e+07,1.277534e+07,...,1.001,0.0,314672.090376,7.649473e+05,14382.095388,1602.053253,5.164207e+06,0.0,7.951841e+06,1.650814e+09
std,1.136369e+09,3.838201,28.421040,461.427463,3.269533e+08,2.648316e+13,6.129057e+08,3.819547e+07,2.062063e+08,3.779474e+07,...,NaN,NaN,NaN,1.307489e+06,NaN,NaN,6.584546e+06,NaN,NaN,0.000000e+00
min,0.000000e+00,-83.622324,-100.000000,-100.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.001,0.0,314672.090376,9.309666e+03,14382.095388,1602.053253,5.082295e+05,0.0,7.951841e+06,1.650814e+09
25%,2.073540e+04,-0.044271,-1.365362,-7.635828,2.299463e+03,2.036916e+06,0.000000e+00,1.551935e+05,3.139112e+02,2.336775e+04,...,1.001,0.0,314672.090376,1.006807e+04,14382.095388,1602.053253,2.836218e+06,0.0,7.951841e+06,1.650814e+09
50%,1.110366e+06,0.001847,-0.359671,-2.968912,1.720931e+05,2.263123e+07,7.224930e+05,2.104848e+06,1.700643e+05,2.014870e+05,...,1.001,0.0,314672.090376,1.082647e+04,14382.095388,1602.053253,5.164207e+06,0.0,7.951841e+06,1.650814e+09
75%,1.735540e+07,0.246862,0.146488,0.075808,2.328181e+06,1.372341e+08,1.307429e+07,1.177762e+07,2.523498e+06,3.851543e+06,...,1.001,0.0,314672.090376,1.142766e+06,14382.095388,1602.053253,7.492195e+06,0.0,7.951841e+06,1.650814e+09
max,1.983024e+10,45.842667,979.424662,16111.256821,8.172051e+09,6.666086e+14,1.112525e+10,2.117430e+08,1.298096e+09,1.950238e+08,...,1.001,0.0,314672.090376,2.274706e+06,14382.095388,1602.053253,9.820184e+06,0.0,7.951841e+06,1.650814e+09


In [ ]:
#@title Dataset Stats { display-mode: "code" }
full_talbe.info(verbose=True)

 ## Total Combined TVL

In [ ]:
  #@title API call { display-mode: "code" }
import requests
import datetime
import time
import json
from google.colab import data_table

import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt

path = "https://api.llama.fi/charts"
#slug = 'aave' 
#General Information table
response = requests.get(path)
output = response.text
DeFi = json.loads(output)
totalLiq_talbe = pd.json_normalize(DeFi)
totalLiq_talbe.head()



In [ ]:
 #@title Save To DeFi Lama Dataset Directory{ display-mode: "code" }
 
table = "deFi_lama_total_TVL"
path = "/content/drive/MyDrive/workspace/pipelines/datasets/defi_lama/output/"
totalLiq_talbe.to_csv(path+ table)

In [ ]:
#@title Load General Information Data to SQLite { display-mode: "code" }


totalLiq_talbe.to_sql(table, conn , if_exists='replace', index=False) # writes to file

 # Test Connection
df = pd.read_sql('select * from {}'.format(table), conn)

df.head()

In [ ]:
conn.close()

# Feature Engineering